In [1]:
import torch
from torch_geometric.data import Data

/home/abir/miniconda3/envs/pygdebias_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# this will form a triangle
edge_index = torch.tensor([[0,1,2],[1,2,0]],dtype=torch.long)

# let's each node has a 2 dimensional feature
x = torch.tensor([[1,2],[2,3],[3,4]],dtype=torch.float)

# let's we have a single graph level classification
y = torch.tensor([1],dtype=torch.float)

In [10]:
data = Data(x=x, edge_index=edge_index, y=y)
print(data)

Data(x=[3, 2], edge_index=[2, 3], y=[1])


## Load previous dataset

In [12]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='dataset', name='Cora')
data = dataset[0]  # There's usually only one graph in citation datasets
print(data)


Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        # First graph convolution + ReLU
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # Second graph convolution
        x = self.conv2(x, edge_index)
        return x


In [14]:
model = GCN(in_channels=dataset.num_node_features, hidden_channels=16, out_channels=dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    # Only use the training nodes for loss calculation
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test():
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        correct = pred[mask].eq(data.y[mask]).sum().item()
        acc = correct / mask.sum().item()
        accs.append(acc)
    return accs  # returns [train_acc, val_acc, test_acc]

for epoch in range(1, 201):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')


Epoch: 010, Loss: 0.7504, Train: 0.9500, Val: 0.7200, Test: 0.7510
Epoch: 020, Loss: 0.1528, Train: 1.0000, Val: 0.7760, Test: 0.7900
Epoch: 030, Loss: 0.0365, Train: 1.0000, Val: 0.7740, Test: 0.7860
Epoch: 040, Loss: 0.0170, Train: 1.0000, Val: 0.7760, Test: 0.7880
Epoch: 050, Loss: 0.0140, Train: 1.0000, Val: 0.7680, Test: 0.7880
Epoch: 060, Loss: 0.0149, Train: 1.0000, Val: 0.7720, Test: 0.7900
Epoch: 070, Loss: 0.0166, Train: 1.0000, Val: 0.7720, Test: 0.7870
Epoch: 080, Loss: 0.0174, Train: 1.0000, Val: 0.7740, Test: 0.7950
Epoch: 090, Loss: 0.0171, Train: 1.0000, Val: 0.7740, Test: 0.7920
Epoch: 100, Loss: 0.0162, Train: 1.0000, Val: 0.7780, Test: 0.7960
Epoch: 110, Loss: 0.0152, Train: 1.0000, Val: 0.7760, Test: 0.7960
Epoch: 120, Loss: 0.0143, Train: 1.0000, Val: 0.7740, Test: 0.7960
Epoch: 130, Loss: 0.0136, Train: 1.0000, Val: 0.7720, Test: 0.7940
Epoch: 140, Loss: 0.0129, Train: 1.0000, Val: 0.7700, Test: 0.7970
Epoch: 150, Loss: 0.0124, Train: 1.0000, Val: 0.7740, Test: 0.